<center><h1>Music & Market: Tracking Global Trends in Song Popularity and Stock Performance</h1>

**Project Description :**
<p>This project aims to explore potential connections between cultural and financial trends by comparing the most popular global songs of 2023 with stock market performance. By analyzing weekly top songs and correlating them with market data (S&P 500, NASDAQ), we hope to uncover any underlying patterns or insights that might link shifts in music tastes to economic factors.</p>

Index (Step-by-Step Process):
1. Project Introduction \
Brief introduction of the goals and scope of the project.

2. Data Collection: Music Data (2023) \
Retrieving weekly top global songs for the year 2023 using reliable sources like Billboard.

3. Data Collection: Stock Market Data (2023) \
Gathering weekly stock market data (S&P 500, NASDAQ) for the same year.

4. Data Cleaning & Preparation \
Formatting and structuring the collected data for analysis.

5. Data Analysis \
Visualizing trends and performing correlation analysis between music and stock market data.

6. Conclusion and Findings \
Summarizing insights and drawing conclusions from the analysis.

7. Website Creation \
Presenting findings in a dynamic, interactive website.

**Step 1** 

In [11]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to get the top 5 songs for a given week
def get_top_songs_for_week(week):
    url = f"https://www.billboard.com/charts/hot-100/{week}/"
    
    # Add headers to mimic a browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    try:
        # Send an HTTP request to get the webpage content
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check if the request was successful
        
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find the top 5 songs by locating the relevant HTML elements
        song_titles = []
        
        # Get all h3 elements for the song titles
        song_tags = soup.find_all("h3", class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only")
        
        # Collect the first 5 song titles
        for song_tag in song_tags[:5]:  # Limit to first 5 songs
            song_title = song_tag.get_text(strip=True)
            song_titles.append(song_title)
        
        # Fill with "N/A" if less than 5 songs are found
        while len(song_titles) < 5:
            song_titles.append("N/A")
        
        return [week] + song_titles  # Return week and top 5 songs
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for week {week}: {e}")
        return [week] + ['Error'] * 5  # Return error message for that week

# Generate the list of weeks in 2023
weeks_in_2023 = [
    "2023-01-07", "2023-01-14", "2023-01-21", "2023-01-28",
    "2023-02-04", "2023-02-11", "2023-02-18", "2023-02-25",
    "2023-03-04", "2023-03-11", "2023-03-18", "2023-03-25",
    "2023-04-01", "2023-04-08", "2023-04-15", "2023-04-22",
    "2023-04-29", "2023-05-06", "2023-05-13", "2023-05-20",
    "2023-05-27", "2023-06-03", "2023-06-10", "2023-06-17",
    "2023-06-24", "2023-07-01", "2023-07-08", "2023-07-15",
    "2023-07-22", "2023-07-29", "2023-08-05", "2023-08-12",
    "2023-08-19", "2023-08-26", "2023-09-02", "2023-09-09",
    "2023-09-16", "2023-09-23", "2023-09-30", "2023-10-07",
    "2023-10-14", "2023-10-21", "2023-10-28", "2023-11-04",
    "2023-11-11", "2023-11-18", "2023-11-25", "2023-12-02",
    "2023-12-09", "2023-12-16", "2023-12-23", "2023-12-30"
]

# Prepare CSV file to save the data
with open("top_5_songs_2023.csv", mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(["Date", "Top 1 Song", "Top 2 Song", "Top 3 Song", "Top 4 Song", "Top 5 Song"])
    
    # Fetch and write the top 5 songs for each week in 2023
    for week in weeks_in_2023:
        top_songs = get_top_songs_for_week(week)
        writer.writerow(top_songs)

print("Data has been written to 'top_5_songs_2023.csv'")


Data has been written to 'top_5_songs_2023.csv'


**Step 2**

In [13]:
import pandas as pd

# Load the CSV file
file_path = 'top_5_songs_2023.csv'  # Change this to your file path
songs_df = pd.read_csv(file_path)

# Defining a simple mood dictionary for the songs
mood_dict = {
    "Rockin' Around The Christmas Tree": "Happy",
    "Jingle Bell Rock": "Happy",
    "Last Christmas": "Romantic",
    "A Holly Jolly Christmas": "Happy",
    "It's The Most Wonderful Time Of The Year": "Happy",
    "Unholy": "Energetic",
    "Kill Bill": "Sad",
    "Anti-Hero": "Sad",
    "I'm Good (Blue)": "Energetic",
    "Creepin'": "Sad",
    "Rich Flex": "Energetic",
    "Die For You": "Romantic"
}

# Function to calculate the overall mood for a week, excluding "Neutral"
def calculate_mood_no_neutral(row):
    # Get the mood for each song, skip songs not in the dictionary
    moods = [mood_dict.get(song) for song in row[1:] if mood_dict.get(song)]
    if moods:  # If there are valid moods, return the most frequent mood
        mood_counts = pd.Series(moods).value_counts()
        return mood_counts.idxmax()
    return "Undefined"  # Fallback if no valid moods

# Apply the function to each row and create a new column "Overall Mood"
songs_df['Overall Mood'] = songs_df.apply(calculate_mood_no_neutral, axis=1)

# Save the updated DataFrame to a new CSV file
output_file_path = 'top_5_songs_2023_with_mood_no_neutral.csv'
songs_df.to_csv(output_file_path, index=False)

print(f"CSV file saved as {output_file_path}")


CSV file saved as top_5_songs_2023_with_mood_no_neutral.csv


**Step 3**

In [12]:
import yfinance as yf
import pandas as pd

# Dates for each week in 2023
weeks_in_2023 = [
    "2023-01-07", "2023-01-14", "2023-01-21", "2023-01-28",
    "2023-02-04", "2023-02-11", "2023-02-18", "2023-02-25",
    "2023-03-04", "2023-03-11", "2023-03-18", "2023-03-25",
    "2023-04-01", "2023-04-08", "2023-04-15", "2023-04-22",
    "2023-04-29", "2023-05-06", "2023-05-13", "2023-05-20",
    "2023-05-27", "2023-06-03", "2023-06-10", "2023-06-17",
    "2023-06-24", "2023-07-01", "2023-07-08", "2023-07-15",
    "2023-07-22", "2023-07-29", "2023-08-05", "2023-08-12",
    "2023-08-19", "2023-08-26", "2023-09-02", "2023-09-09",
    "2023-09-16", "2023-09-23", "2023-09-30", "2023-10-07",
    "2023-10-14", "2023-10-21", "2023-10-28", "2023-11-04",
    "2023-11-11", "2023-11-18", "2023-11-25", "2023-12-02",
    "2023-12-09", "2023-12-16", "2023-12-23", "2023-12-30"
]

# Initialize a list to store results
results = []

# Loop through each date
for week in weeks_in_2023:
    try:
        # Fetch data for the specific date
        data = yf.download("ACWI", start=week, end=pd.to_datetime(week) + pd.DateOffset(7), interval="1d")
        
        # Check if data is not empty
        if not data.empty:
            # Get the close price from the last day of the week
            close_price = data['Close'].iloc[-1]  # Fetching the last available close price
            results.append({'Date': week, 'Close Price': close_price})
        else:
            results.append({'Date': week, 'Close Price': None})  # No data available
    except Exception as e:
        print(f"Error fetching data for {week}: {e}")
        results.append({'Date': week, 'Close Price': None})  # Handle error

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save data to a CSV file
df.to_csv('acwi_weekly_prices_2023.csv', index=False)

print("Weekly ACWI stock prices saved successfully!")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Weekly ACWI stock prices saved successfully!
